In [1]:
from paz.backend.image import load_image, show_image, write_image
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr
from paz.backend.image.image import crop_image
from paz.processors.detection import CropBoxes2D
import numpy as np
from pathlib import Path
from paz.abstract import SequentialProcessor
from retinaface import RetinaFace
from paz.abstract.messages import Box2D

In [2]:
class process_front_images():
    def __init__(self):
        self.folder_path = Path('DMD\s3_rgb_face').__str__()
        self.result = np.array([[0, 0]]) # 2D array with element [frame_number, emotion]
        self.frame_offset = 10
        self.number_images = 100
    def get_emotion(self):
        for frame_number in range(1, 1 + self.number_images + self.frame_offset): # starts from 0 + frame_offset
            path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            print('Boxes: ', boxes2D, type(boxes2D))
            if (len(boxes2D) == 0):
                self.result = np.append(self.result, [[frame_number, 'nofacedetected']], axis=0)
                continue
                
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                print('Indices to remove', indices_excluding_max)
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D)            
            
            cropped_images = crop(image, boxes2D)
            #show_image(cropped_images[0])
            classify = MiniXceptionFER()
            emotion = classify(cropped_images[0])['class_name']
            print([frame_number, emotion])
            self.result = np.append(self.result, [[frame_number, emotion]], axis=0)
        print('----------------Done getting emotion------------------')
        return self.result
    
front_image_processor = process_front_images()
emotion_result = front_image_processor.get_emotion()
print('Front face emotion labeling: ', emotion_result, emotion_result.shape)

--------Frame 1------
Boxes:  [Box2D(495, 302, 708, 553, 0, None)] <class 'list'>
[1, 'sad']
--------Frame 2------
Boxes:  [Box2D(494, 303, 708, 554, 0, None)] <class 'list'>
[2, 'sad']
--------Frame 3------
Boxes:  [Box2D(494, 304, 707, 557, 0, None)] <class 'list'>
[3, 'sad']
--------Frame 4------
Boxes:  [Box2D(497, 302, 715, 566, 0, None)] <class 'list'>
[4, 'sad']
--------Frame 5------
Boxes:  [Box2D(497, 302, 715, 566, 0, None)] <class 'list'>
[5, 'sad']
--------Frame 6------
Boxes:  [Box2D(496, 306, 716, 568, 0, None)] <class 'list'>
[6, 'sad']
--------Frame 7------
Boxes:  [Box2D(497, 308, 715, 569, 0, None)] <class 'list'>
[7, 'sad']
--------Frame 8------
Boxes:  [Box2D(496, 308, 717, 572, 0, None)] <class 'list'>
[8, 'sad']
--------Frame 9------
Boxes:  [Box2D(497, 308, 717, 575, 0, None)] <class 'list'>
[9, 'sad']
--------Frame 10------
Boxes:  [Box2D(497, 309, 718, 577, 0, None)] <class 'list'>
[10, 'sad']
--------Frame 11------
Boxes:  [Box2D(497, 307, 717, 576, 0, None)] 

Boxes:  [Box2D(507, 308, 726, 581, 0, None)] <class 'list'>
[86, 'sad']
--------Frame 87------
Boxes:  [Box2D(507, 307, 725, 582, 0, None)] <class 'list'>
[87, 'sad']
--------Frame 88------
Boxes:  [Box2D(508, 308, 725, 581, 0, None)] <class 'list'>
[88, 'sad']
--------Frame 89------
Boxes:  [Box2D(507, 308, 725, 580, 0, None)] <class 'list'>
[89, 'sad']
--------Frame 90------
Boxes:  [Box2D(508, 306, 724, 579, 0, None)] <class 'list'>
[90, 'sad']
--------Frame 91------
Boxes:  [Box2D(506, 307, 723, 577, 0, None)] <class 'list'>
[91, 'sad']
--------Frame 92------
Boxes:  [Box2D(495, 310, 716, 584, 0, None)] <class 'list'>
[92, 'neutral']
--------Frame 93------
Boxes:  [Box2D(506, 304, 722, 577, 0, None)] <class 'list'>
[93, 'sad']
--------Frame 94------
Boxes:  [Box2D(505, 305, 722, 574, 0, None)] <class 'list'>
[94, 'sad']
--------Frame 95------
Boxes:  [Box2D(506, 305, 722, 573, 0, None)] <class 'list'>
[95, 'sad']
--------Frame 96------
Boxes:  [Box2D(506, 304, 722, 573, 0, None)] <

In [3]:
from paz.backend.image import write_image
class process_side_images():
    def __init__(self):
        self.folder_path = Path('DMD\s3_rgb_body').__str__()
        self.result = np.array([['0', '0']]) # path format?
        self.frame_offset = 10
        self.number_images = 100
    def get_face_image(self):
        for frame_number in range(1, 1 + self.number_images):
            path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
            emotion = emotion_result[frame_number + self.frame_offset][1]
            
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            print('Boxes: ', boxes2D, type(boxes2D))
            if (len(boxes2D) == 0):
                self.result = np.append(self.result, [[frame_number, '0']], axis=0) # 0 indicates no face detected
                continue
                
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                print([frame_number, emotion])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                print('Indices to remove', indices_excluding_max)
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D)            
            
            if (emotion == 'nofacedetected'):
                    self.result = np.append(self.result, [[frame_number, '0']], axis=0)
            
            else:   # both dataset have face detected for this frame number              
                cropped_images = crop(image, boxes2D)
                #show_image(cropped_images[0])
                result_path = 'results\img.%(number)06d.png'% {'number': frame_number}
                write_image(result_path, cropped_images[0])
                self.result = np.append(self.result, [[result_path, emotion]], axis=0)
side_processor = process_side_images()
side_processor.get_face_image()

--------Frame 1------
Boxes:  [Box2D(393, 25, 541, 238, 0, None)] <class 'list'>
--------Frame 2------
Boxes:  [Box2D(395, 28, 543, 238, 0, None)] <class 'list'>
--------Frame 3------
Boxes:  [Box2D(397, 28, 543, 239, 0, None)] <class 'list'>
--------Frame 4------
Boxes:  [Box2D(398, 30, 543, 240, 0, None)] <class 'list'>
--------Frame 5------
Boxes:  [Box2D(399, 29, 544, 243, 0, None)] <class 'list'>
--------Frame 6------
Boxes:  [Box2D(399, 29, 544, 243, 0, None)] <class 'list'>
--------Frame 7------
Boxes:  [Box2D(399, 30, 543, 243, 0, None)] <class 'list'>
--------Frame 8------
Boxes:  [Box2D(400, 32, 544, 246, 0, None)] <class 'list'>
--------Frame 9------
Boxes:  [Box2D(402, 33, 545, 245, 0, None)] <class 'list'>
--------Frame 10------
Boxes:  [Box2D(402, 34, 545, 246, 0, None)] <class 'list'>
--------Frame 11------
Boxes:  [Box2D(403, 34, 546, 248, 0, None)] <class 'list'>
--------Frame 12------
Boxes:  [Box2D(403, 33, 547, 249, 0, None)] <class 'list'>
--------Frame 13------
Bo

In [4]:
side_processor.result
np.savetxt("pseudo_labelling_results.csv", side_processor.result, delimiter=",", fmt='%s')

### Code for detecting faces using PAZ

Poor detection performance
    4 out of 5 images were detected wrongly - boxes not appears around the face

In [5]:
path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            detect = HaarCascadeFrontalFace(draw=False)
            crop = CropBoxes2D()
            inference = detect(image)
            boxes2D = inference['boxes2D']
            print('Boxes: ', boxes2D, type(boxes2D))

IndentationError: unexpected indent (1078660620.py, line 2)